In [ ]:
import os
import pandas as pd
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Select, CustomJS
from bokeh.layouts import column
from bokeh.io import output_file, show
from bokeh.palettes import Category10

In [64]:
input_folder = '../../fred_input_files/dominance_data'
dataframes = {}

# Define the columns you expect in your DataFrame
expected_columns = ['date', 'week', 'Alpha', 'Gamma', 'Mu', 'Omicron', 'OmicronBAX']

for file in os.listdir(input_folder):
    if file.endswith(".csv"):  # Make sure to process only CSV files
        file_path = os.path.join(input_folder, file)
        df_dom = pd.read_csv(file_path)
        
        # Convert the 'date' column to datetime
        df_dom['date'] = pd.to_datetime(df_dom['date'])

        # Reindex the DataFrame to have the expected columns, filling with 0 if they don't exist
        df_dom = df_dom.reindex(columns=expected_columns, fill_value=0)

        # Store the DataFrame in the dictionary
        dataframes[file] = df_dom


In [67]:
# Convert DataFrames to ColumnDataSource, which can be used in Bokeh
sources = {name: ColumnDataSource(df) for name, df in dataframes.items()}

# Initial ColumnDataSource
initial_source = sources[os.listdir(input_folder)[0]]

# Create the figure with datetime x-axis
p = figure(x_axis_type='datetime', title="Dominance Data Over Time", width=800, height=400)

# Assuming 'date' is the name of your datetime column
date_column = 'date'

# List of columns to plot
columns_to_plot = ['Alpha', 'Gamma', 'Mu', 'Omicron', 'OmicronBAX']

# Check if we have enough colors, otherwise cycle the palette
if len(columns_to_plot) > len(Category10):
    colors = Category10 * (len(columns_to_plot) // len(Category10) + 1)
else:
    colors = Category10[10][:len(columns_to_plot)]

# Plot line and scatter for each column with a unique color
for i, column_name in enumerate(columns_to_plot):
    color = colors[i]
    p.line(x=date_column, y=column_name, source=initial_source, line_width=2, color=color, alpha=0.8, legend_label=column_name)
    p.scatter(x=date_column, y=column_name, source=initial_source, size=8, color=color, legend_label=column_name)


# Add legend customization here if needed
p.legend.location = "top_left"
p.legend.click_policy = "hide"


# Select widget with JavaScript callback
select = Select(title="Choose dataset:", value=os.listdir(input_folder)[0], options=list(sources.keys()))
select.js_on_change('value', CustomJS(args=dict(sources=sources, plot=p), code="""
    const selected = cb_obj.value;
    const new_source = sources[selected];
    plot.renderers[0].data_source.data = new_source.data;
    plot.renderers[0].data_source.change.emit();
"""))

# Layout
layout = column(select, p)

# Output to HTML
output_file('../output_files/timeseries_plot.html')
show(layout)